In [2]:
import constants
BASE = constants.BASE
SIZE = constants.SIZE
ADDITIVE_IDENTITY = constants.ADDITIVE_IDENTITY

def add_two_num(arr1, arr2):
    """
    Add two arrays of numbers.
    """
    if len(arr1) <= len(arr2):
        arr1, arr2 = arr2, arr1
    arr2 = arr2 + [0]*(len(arr1) - len(arr2))
    result = []
    carry = 0
    for i in range(len(arr1)):
        sum = arr1[i] + arr2[i] + carry
        val = sum % BASE
        if sum % BASE == 0:
            val = BASE
        result.append(val)
        carry = (sum - val) // BASE
    if carry != 0:
        result.append(carry)
    return result

In [3]:
def print_number(arr):
    """
    Print the number in the array.
    """
    for i in range(len(arr)):
        print(arr[i], end="")
    print()

In [4]:
a = [3, 4, 4, 2]
b = [1, 2, 3, 4, 5, 4]
print_number(add_two_num(a, ADDITIVE_IDENTITY))
print_number(ADDITIVE_IDENTITY)
print_number(add_two_num(ADDITIVE_IDENTITY, b))
print_number(add_two_num(ADDITIVE_IDENTITY, [1]))

3442766666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666
7666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666
1234547666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666
1766666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666


In [5]:
def get_additive_inverse(arr):
    if len(arr) < SIZE:
        arr = arr + [0]*(SIZE - len(arr))
    result = []
    carry = 0
    for i in range(SIZE):
        x = ADDITIVE_IDENTITY[i] - arr[i] - carry
        if x <= 0:
            x = x + BASE
            carry = 1
        result.append(x)
    return result

def subtract(arr1, arr2):
    return add_two_num(arr1, get_additive_inverse(arr2))
print_number(subtract(ADDITIVE_IDENTITY, [1]))

66666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666661


In [6]:
from tokenize import Double

def newton_raphson_square_root(n) -> Double:
    """
    Find the square root of n using Newton-Raphson method.
    """
    x = n
    while True:
        y = (x + n/x) / 2
        if y == x:
            return y
        x = y
print(newton_raphson_square_root(9.2))

3.03315017762062


In [19]:
def convert_to_p_adic(n, arr = []): #optional parameter
    """
    Convert a number to p-adic number.
    """
    if n <= 0:
        return add_two_num(ADDITIVE_IDENTITY, arr)
    if n % BASE == 0:
        rem = (n-BASE) // BASE
        return convert_to_p_adic(rem, arr + [BASE])
    num = n % BASE
    return convert_to_p_adic(n // BASE, arr + [num])
print_number(convert_to_p_adic(100))

2717666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666


In [8]:
def convert_to_decimal(arr):
    """
    Convert a p-adic number to decimal.
    """
    result = 0
    for i in range(len(arr)):
        result = result + arr[i] * BASE**i
    return result
def convert_to_base(arr):#convert to ordinary p-adic number with digits 0, 1, 2, 3,...., BASE-1
    """
    Convert a p-adic number to base.
    """
    n = convert_to_decimal(arr)
    result = []
    while n > 0:
        result.append(n % BASE)
        n = n // BASE
    return result[::1] #reverse the list as, p-adic number represented in base is in reverse order
print(convert_to_decimal(ADDITIVE_IDENTITY))
print_number(convert_to_base(ADDITIVE_IDENTITY))

3234476509624757991344647769100216810857203198904625400933895331391691459636928060001
00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000001


In [9]:
def multiplication_by_digit(arr, d):
    """
    Multiply a p-adic number by a digit.
    """
    result = []
    carry = 0
    for i in range(len(arr)):
        sum = arr[i] * d + carry
        val = sum % BASE
        carry = (sum - val) // BASE
        if val == 0:
            val += BASE
            carry -= 1
        result.append(val)
    return result

n = add_two_num(ADDITIVE_IDENTITY, [2,5])
m = multiplication_by_digit(n, 2) #will be 74 for base 7, 54 for base 5
print_number(multiplication_by_digit(n, 2))

4317666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666


In [10]:
def multiplication(arr1, arr2):
    """
    Multiply two p-adic numbers.
    """
    result = []
    for i in range(len(arr2)):
        #adding 0 to shift the array to the right
        #that means multiply the number by BASE like in ordinary multiplication
        result = add_two_num(result, [0] * i + multiplication_by_digit(arr1, arr2[i]))
    return result[: min(SIZE, len(result))] # return only first SIZE digits
m = [2, 1] # 9 in base 7
n = [1, 2] # 15 in base 7
p = multiplication(m, n) # multiplication of 9 and 15 in base 7
print(convert_to_decimal(p))
print_number(p)
#now see the infinite representation of multiplication of 9 and 15 in base 7
m = add_two_num(ADDITIVE_IDENTITY, m)
n = add_two_num(ADDITIVE_IDENTITY, n)
p = multiplication(m, n)
print_number(p)

135
252
2527666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666


In [33]:
from multiprocessing.dummy import Array
import constants
MULTIPLICATIVE_IDENTITY = constants.MULTIPLICATIVE_IDENTITY
def get_multiplicative_inverse(arr, res = [], pos = 0) -> Array:
    if pos == SIZE: #we are done
        return res
    if pos == 0: #first digit case
        res = [0] * SIZE
    if len(arr) < SIZE:
        arr = add_two_num(arr, ADDITIVE_IDENTITY)
    for i in range(1, BASE+1):
        res[pos] = i
        prod = multiplication(arr, res)
        if MULTIPLICATIVE_IDENTITY[pos] == prod[pos]:
            return get_multiplicative_inverse(arr, res, pos + 1)
    return [-1] * SIZE
a = get_multiplicative_inverse([2, 7, 1]) #100 in base 7
b = convert_to_p_adic(100)
print_number(a)
print_number(b)
print_number(multiplication(a, b))

4366336633663366336633663366336633663366336633663366336633663366336633663366336633663366336633663366
2717666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666
1766666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666


In [36]:
def division(arr1, arr2):
    return multiplication(arr1, get_multiplicative_inverse(arr2))
a = convert_to_p_adic(5)
b = convert_to_p_adic(3)
c = division(a, b)
print_number(c)
d = multiplication(c, b)
print_number(d)

4222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222
5766666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666
